In [26]:
!wget https://pilehvar.github.io/wic/package/WiC_dataset.zip

--2023-05-21 06:37:39--  https://pilehvar.github.io/wic/package/WiC_dataset.zip
Resolving pilehvar.github.io (pilehvar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to pilehvar.github.io (pilehvar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275984 (270K) [application/zip]
Saving to: ‘WiC_dataset.zip.1’

WiC_dataset.zip.1   100%[===================>] 269.52K  --.-KB/s    in 0.005s  

2023-05-21 06:37:39 (49.5 MB/s) - ‘WiC_dataset.zip.1’ saved [275984/275984]



In [27]:
import pandas as pd
import numpy as np
import os
import io
import spacy
nlp = spacy.load("en_core_web_sm")

In [28]:
directory = os.makedirs( '/content/WIC_data', exist_ok=True )

In [29]:
dev = os.makedirs( '/content/WIC/dev', exist_ok=True )
train = os.makedirs( '/content/WIC/train', exist_ok=True )
test = os.makedirs( '/content/WIC/test', exist_ok=True )


In [30]:
#path = "/content/dev/dev.data.txt"

In [31]:
#directory = path.split('/')[2]

In [32]:
!pip install spacy  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
from zipfile import ZipFile
with ZipFile('WiC_dataset.zip', 'r') as WiC:
   WiC.extractall('/content/WiC_data')

In [34]:
ds = os.listdir('/content/WiC_data')

In [36]:
ds.remove('README.txt')

In [37]:
pathss = []
for i in ds:
  pathss.append('/content/WiC_data/' + i + '/' + i+'.data.txt')


In [38]:
path_jud = []
for i in ds:
  path_jud.append('/content/WiC_data/' + i + '/' + i+'.gold.txt')

In [39]:
df_all = pd.DataFrame()
for i in pathss:
  df_3 = pd.read_csv(i, sep = '\t', names=["lemma", "pos", "index", "sent1", "sent2"], encoding = 'UTF-8')
  df_3['dataset'] = i.split('/')[3]
  df_all = pd.concat([df_all, df_3])

In [40]:
df_all_labels = pd.DataFrame()
for i in path_jud:
  tmp = pd.read_csv(i, sep = '\t', names = ['judgment'], encoding = 'UTF-8')
  tmp['dataset'] = i.split('/')[3]
  df_all_labels = pd.concat([df_all_labels, tmp])

In [42]:
x= df_all['index'].tolist()


In [43]:
df_dash = df_all[['lemma', 'sent1', 'sent2', 'dataset']]

In [44]:
df1 = df_dash[['lemma', 'sent1', 'dataset']]
df1.columns =  ['word', 'sent', 'dataset']
df1["index"] = df1.groupby("word").cumcount()*2

In [45]:
df2 =  df_dash[['lemma', 'sent2', 'dataset']]
df2.columns =  ['word', 'sent', 'dataset']
df2["index"] = df2.groupby("word").cumcount()*2+1

In [46]:
df_final = pd.concat([df1, df2])
df_final= df_final.sort_values(by = ['word', 'index'], ascending = [True, True]) 

In [47]:
df_final["identifier"] = df_final["word"]+"-"+df_final["index"].astype(str)


In [55]:
df_odd = df_final[1::2]
df_odd.columns = ["word2", "sent2", 'dataset1',"index2", "identifier2"]
df_even = df_final[::2]
df_even.columns = ["word1", "sent1", 'dataset2', "index1", "identifier1"]
df_final_ = pd.concat([df_even, df_odd], axis=1)


In [56]:
df_final_ = df_final_[["word2", "sent2", "identifier2", "sent1", "identifier1", 'dataset1','index1', 'index2']]
df_final_ = df_final_.rename(columns={"word2":"lemma"})


In [58]:
df_final_["judgment"] = df_all_labels["judgment"].values.tolist()

In [60]:
df_final_ = df_final_[['lemma', 'identifier2', 'identifier1', 'judgment', 'dataset1']]

In [62]:
df_final_["annotator"] = "gold"
df_final_['comment'] = " "

<ipython-input-62-9f1458d95f3a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_["annotator"] = "gold"
<ipython-input-62-9f1458d95f3a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final_['comment'] = " "


In [64]:
df_final_ = df_final_[["identifier1", "identifier2", "annotator", "judgment", "comment", "lemma", 'dataset1']]

In [65]:
df_final_ = df_final_.reset_index(drop =True)

In [67]:
ind1 = []
ind2 = []
for i in x:
    ind1.append(i.split('-')[0])
    ind2.append(i.split('-')[1])

In [68]:
df_all.loc[:,'ind1'] = ind1
df_all.loc[:,'ind2'] = ind2


In [70]:
df1 = df_all[['lemma', 'pos', 'sent1', 'ind1', 'dataset']]
df1.columns =  ['lemma','pos', 'context_tokenized','ind', 'dataset']


In [71]:
df2 = df_all[['lemma', 'pos', 'sent2', 'ind2', 'dataset']]
df2.columns =  ['lemma','pos', 'context_tokenized','ind', 'dataset']

In [73]:
df_final_uses = pd.concat([df1, df2])
df_final_uses = df_final_uses.sort_values(by = ['lemma'], ascending = True)


In [74]:
df_final_uses = df_final_uses.reset_index(drop =True)

In [76]:
def get_indice(sent, indice):
  tag = ''
  indice = int(indice)
  tok = sent.split(" ")
  #print(tok)
  for i in range(len(tok)):
    if i == indice:
      tag = tok[i]
  return str(sent.find(tag))+":"+str(sent.find(tag)+len(tag))

In [78]:
df_final_uses["indexes_target_token"] = df_final_uses.apply(lambda x: get_indice(x.context_tokenized, x.ind), axis=1)


In [79]:
def get_indices_of_sent(sentence):
    return "0:"+str(len(sentence))

In [80]:
df_final_uses["indexes_target_sentence"] = df_final_uses["context_tokenized"].apply(get_indices_of_sent)

In [81]:
def get_len_tok(sentence):
  return "0:"+str(len(sentence.split(" ")))

In [82]:
df_final_uses['indexes_target_sentence_tokenized'] = df_final_uses["context_tokenized"].apply(get_len_tok)


In [83]:
lem = ''
def lemmatizr(sent):
  doc = nlp(sent)
  for i in doc:
    lem += i.lemma_
    return lem

In [84]:
df_final_uses['context_lemmatized'] = df_final_uses["context_tokenized"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))

In [85]:
df_final_uses["index"] = df_final_uses.groupby("lemma").cumcount()

In [87]:
df_final_uses['context_pos'] = df_final_uses["context_tokenized"].apply(lambda row: " ".join([w.pos_ for w in nlp(row)]))

In [89]:
df_final_uses['date'] = " "
df_final_uses['grouping'] = 1
df_final_uses['description'] = " "
df_final_uses['context'] = df_final_uses['context_tokenized']
#df_final_dev_uses['identifier'] = ""
df_final_uses['indexes_target_token_tokenized'] = df_final_uses['ind']


In [90]:
df_final_uses = df_final_uses.drop_duplicates().reset_index(drop = True)

In [92]:
df_final_uses["identifier"] = df_final_uses["lemma"]+"-"+df_final_uses["index"].astype(str)

In [98]:
final_df = df_final_uses[['lemma', 'pos', 'date', 'grouping', 'identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'context_tokenized', 'indexes_target_token_tokenized', 'indexes_target_sentence_tokenized', 'context_lemmatized', 'context_pos', 'dataset']]


In [94]:
!git clone https://github.com/Garrafao/WUGs.git

Cloning into 'WUGs'...
remote: Enumerating objects: 1379, done.
remote: Counting objects: 100% (305/305), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 1379 (delta 168), reused 275 (delta 149), pack-reused 1074
Receiving objects: 100% (1379/1379), 1.79 MiB | 17.46 MiB/s, done.
Resolving deltas: 100% (846/846), done.


In [95]:
os.makedirs('/content/WUGs/scripts/misc/data/dwug_en/data/*/', exist_ok=True )

In [99]:
import csv
final_df.to_csv('/content/WUGs/scripts/misc/data/dwug_en/data/*/uses.csv', sep='\t', encoding='utf-8', quoting = csv.QUOTE_NONE, quotechar='')


In [ ]:
%run /content/WUGs/scripts/misc/use2normalize.py /content/WUGs/scripts/misc/data/dwug_en/data/*/uses.csv dwug_en output

In [101]:
final_uses = pd.read_csv('/content/output', delimiter = '\t',quoting =3)


In [102]:
final_uses['context_tokenized'] = final_df['context_tokenized']
final_uses['pos'] = final_df['pos']
final_uses['dataset'] = final_df['dataset']
final_uses['grouping'] = final_df['grouping']
final_uses['lemma'] = final_df['lemma']
final_uses['indexes_target_token_tokenized'] = final_df['indexes_target_token_tokenized']
final_uses['indexes_target_sentence_tokenized'] = final_df['indexes_target_sentence_tokenized']
final_uses['date'] = " "
final_uses['description'] = " "
final_uses['context_lemmatized'] = final_df['context_lemmatized']
final_uses['context_pos'] = final_df['context_pos']


In [103]:
final_uses = final_uses[['lemma', 'pos', 'date', 'grouping', 'identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'context_tokenized', 'indexes_target_token_tokenized', 'indexes_target_sentence_tokenized', 'context_lemmatized', 'context_pos', 'dataset']]


In [110]:
for i in list(final_uses["dataset"].value_counts().index):
    df_temp = final_uses[final_uses["dataset"]==i]
    if not os.path.exists(i):
        os.mkdir(i)
    df_temp.to_csv('/content/WIC/' + i +'/uses.csv',index = False, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONE, quotechar = '')

In [109]:
for i in list(df_final_["dataset1"].value_counts().index):
    df_temp = df_final_[df_final_["dataset1"]==i]
    if not os.path.exists(i):
        os.mkdir(i)
    df_temp.to_csv('/content/WIC/' + i +'/judgments.csv',index = False, sep='\t', encoding='utf-8', quoting=csv.QUOTE_NONE, quotechar = '')

In [ ]:
#c = final_df.context.tolist()

In [ ]:
#y = []
#for m in c:
 # y.append(m.split(" "))

In [ ]:
#lengthh = []
#for i in y:
 #   lengthh.append(len(i))

In [ ]:
#final_df.loc[:,'lengthh'] = lengthh

In [ ]:
#d = final_df.context_pos.tolist()

In [ ]:
#z = []
#for m in d:
 # z.append(m.split(" "))

In [ ]:
#lengthh2 = []
#for i in z:
 #   lengthh2.append(len(i))

In [ ]:
#final_df.loc[:,'lengthh2'] = lengthh2